In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

%matplotlib inline

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [4]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [5]:
movie_genres = [change_string(g) for g in movies.genres.values]

In [6]:
movie_genres[:10]

['Adventure Animation Children Comedy Fantasy',
 'Adventure Children Fantasy',
 'Comedy Romance',
 'Comedy Drama Romance',
 'Comedy',
 'Action Crime Thriller',
 'Comedy Romance',
 'Adventure Children',
 'Action',
 'Action Adventure Thriller']

In [7]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)

In [9]:
X_train_counts

<9742x20 sparse matrix of type '<class 'numpy.int64'>'
	with 22084 stored elements in Compressed Sparse Row format>

In [8]:
count_vect.vocabulary_

{'adventure': 1,
 'animation': 2,
 'children': 3,
 'comedy': 4,
 'fantasy': 8,
 'romance': 15,
 'drama': 7,
 'action': 0,
 'crime': 5,
 'thriller': 17,
 'horror': 10,
 'mystery': 13,
 'scifi': 16,
 'war': 18,
 'musical': 12,
 'documentary': 6,
 'imax': 11,
 'western': 19,
 'filmnoir': 9,
 'nogenreslisted': 14}

In [10]:
X_train_counts.toarray()

array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [76]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [13]:
## Альтернативный способ
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(movie_genres)

In [14]:
tfidf_vectorizer.vocabulary_

{'adventure': 1,
 'animation': 2,
 'children': 3,
 'comedy': 4,
 'fantasy': 8,
 'romance': 15,
 'drama': 7,
 'action': 0,
 'crime': 5,
 'thriller': 17,
 'horror': 10,
 'mystery': 13,
 'scifi': 16,
 'war': 18,
 'musical': 12,
 'documentary': 6,
 'imax': 11,
 'western': 19,
 'filmnoir': 9,
 'nogenreslisted': 14}

In [17]:
X_train_tfidf.toarray()

array([[0.        , 0.41684567, 0.51622547, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.51236121, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.57860574, 0.        , 0.81560738, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [20]:
neigh = NearestNeighbors(n_neighbors=7, n_jobs=-1, metric='euclidean') 
neigh.fit(X_train_tfidf)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='euclidean',
         metric_params=None, n_jobs=-1, n_neighbors=7, p=2, radius=1.0)

In [22]:
test = change_string("Adventure|Comedy|Fantasy|Crime")

X_tfidf2 = tfidf_vectorizer.transform([test])

res = neigh.kneighbors(X_tfidf2, return_distance=True)

In [23]:
res

(array([[0.42079615, 0.53300564, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608]]),
 array([[6774, 9096, 5636, 6723, 3376, 7496, 9717]]))

In [24]:
movies.iloc[res[1][0]]

,movieId,title,genres
6774,60074,Hancock (2008),Action|Adventure|Comedy|Crime|Fantasy
9096,143559,L.A. Slasher (2015),Comedy|Crime|Fantasy
5636,27368,Asterix & Obelix: Mission Cleopatra (Astérix &...,Adventure|Comedy|Fantasy
6723,58972,Nim's Island (2008),Adventure|Comedy|Fantasy
3376,4591,Erik the Viking (1989),Adventure|Comedy|Fantasy
7496,82854,Gulliver's Travels (2010),Adventure|Comedy|Fantasy
9717,188833,The Man Who Killed Don Quixote (2018),Adventure|Comedy|Fantasy


In [13]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [51]:
movies_with_tags = movies.join(tags.set_index('movieId'), on='movieId')

In [52]:
movies_with_tags.head()

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09


In [53]:
movies_with_tags[movies_with_tags.title == 'Toy Story (1995)']

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09


In [54]:
movies_with_tags.tag.unique()

array(['pixar', 'fun', 'fantasy', ..., 'star wars', 'gintama', 'remaster'],
      dtype=object)

In [55]:
movies_with_tags.dropna(inplace=True)

In [56]:
movies_with_tags.title.unique().shape

(1572,)

In [57]:
tag_strings = []
movies = []

for movie, group in tqdm_notebook(movies_with_tags.groupby('title')):
    tag_strings.append(' '.join([str(s).replace(' ', '').replace('-', '') for s in group.tag.values]))
    movies.append(movie)

In [58]:
tag_strings[:5]

['artistic Funny humorous inspiring intelligent quirky romance ZooeyDeschanel',
 'lawyers',
 'creepy suspense',
 'Shakespearesortof',
 'dogs remake']

In [59]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(tag_strings)

In [60]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [61]:
neigh = NearestNeighbors(n_neighbors=10, n_jobs=-1, metric='manhattan') 
neigh.fit(X_train_tfidf)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='manhattan',
         metric_params=None, n_jobs=-1, n_neighbors=10, p=2, radius=1.0)

In [62]:
for i in range(len(movies)):
    if 'Magnolia (1999)' == movies[i]:
        print(i)

822


In [63]:
tag_strings[822]

'L.A.'

In [64]:
test = change_string('pixar pixar fun')

predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)

res = neigh.kneighbors(X_tfidf2, return_distance=True)

In [65]:
res

(array([[0., 0., 1., 1., 1., 1., 1., 1., 1., 1.]]),
 array([[661, 822, 947, 955, 954, 953, 951, 950, 949, 959]]))

In [66]:
for i in res[1][0]:
    print(movies[i])

In a Lonely Place (1950)
Magnolia (1999)
Neon Genesis Evangelion: Death & Rebirth (Shin seiki Evangelion Gekijô-ban: Shito shinsei) (1997)
Night and Day (1946)
Nicholas Nickleby (2002)
Niagara (1953)
Never Been Kissed (1999)
Network (1976)
Net, The (1995)
Night of the Hunter, The (1955)


In [67]:
movies

['(500) Days of Summer (2009)',
 '...And Justice for All (1979)',
 '10 Cloverfield Lane (2016)',
 '10 Things I Hate About You (1999)',
 '101 Dalmatians (1996)',
 '101 Dalmatians (One Hundred and One Dalmatians) (1961)',
 '11\'09"01 - September 11 (2002)',
 '12 Angry Men (1957)',
 '127 Hours (2010)',
 '13 Going on 30 (2004)',
 '2001: A Space Odyssey (1968)',
 '21 Grams (2003)',
 '25th Hour (2002)',
 '28 Days Later (2002)',
 '39 Steps, The (1935)',
 '3:10 to Yuma (2007)',
 '40-Year-Old Virgin, The (2005)',
 '400 Blows, The (Les quatre cents coups) (1959)',
 '42 Up (1998)',
 '84 Charing Cross Road (1987)',
 '8MM (1999)',
 'A Million Ways to Die in the West (2014)',
 'A Pigeon Sat on a Branch Reflecting on Existence (2014)',
 'A Story of Children and Film (2013)',
 'A.I. Artificial Intelligence (2001)',
 'About a Boy (2002)',
 'Accused, The (1988)',
 "Adam's Rib (1949)",
 'Addams Family Values (1993)',
 'Addams Family, The (1991)',
 'Adventures of Priscilla, Queen of the Desert, The (1994)

In [69]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [78]:
tfidfs_on_genre = X_train_tfidf.toarray()

In [82]:
tfidfs_on_genre.shape

(9742, 20)

In [85]:
for x in range(tfidfs_on_genre.shape[1]):
    col_name = 'g{}'.format(x)
    movies[col_name] = pd.Series(tfidfs_on_genre[:, x])
        

In [86]:
movies

,movieId,title,genres,g0,g1,g2,g3,g4,g5,g6,...,g10,g11,g12,g13,g14,g15,g16,g17,g18,g19
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.000000,0.416846,0.516225,0.504845,0.267586,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0.000000,0.512361,0.000000,0.620525,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.000000,0.000000,0.000000,0.000000,0.570915,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.821009,0.000000,0.000000,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0.000000,0.000000,0.000000,0.000000,0.505015,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.726241,0.000000,0.000000,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
5,6,Heat (1995),Action|Crime|Thriller,0.549328,0.000000,0.000000,0.000000,0.000000,0.635947,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.542042,0.0,0.0
6,7,Sabrina (1995),Comedy|Romance,0.000000,0.000000,0.000000,0.000000,0.570915,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.821009,0.000000,0.000000,0.0,0.0
7,8,Tom and Huck (1995),Adventure|Children,0.000000,0.636699,0.000000,0.771112,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
8,9,Sudden Death (1995),Action,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
9,10,GoldenEye (1995),Action|Adventure|Thriller,0.553065,0.629522,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.545730,0.0,0.0


In [92]:
def extract_year(s):
    try:
        return int(s[-5:-1])
    except:
        return 0

In [94]:
movies['year'] = movies['title'].apply(extract_year)

In [95]:
movies

,movieId,title,genres,g0,g1,g2,g3,g4,g5,g6,...,g11,g12,g13,g14,g15,g16,g17,g18,g19,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.000000,0.416846,0.516225,0.504845,0.267586,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,0.000000,0.512361,0.000000,0.620525,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,0.000000,0.000000,0.000000,0.000000,0.570915,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.821009,0.000000,0.000000,0.0,0.0,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0.000000,0.000000,0.000000,0.000000,0.505015,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.726241,0.000000,0.000000,0.0,0.0,1995
4,5,Father of the Bride Part II (1995),Comedy,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,1995
5,6,Heat (1995),Action|Crime|Thriller,0.549328,0.000000,0.000000,0.000000,0.000000,0.635947,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.542042,0.0,0.0,1995
6,7,Sabrina (1995),Comedy|Romance,0.000000,0.000000,0.000000,0.000000,0.570915,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.821009,0.000000,0.000000,0.0,0.0,1995
7,8,Tom and Huck (1995),Adventure|Children,0.000000,0.636699,0.000000,0.771112,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,1995
8,9,Sudden Death (1995),Action,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,1995
9,10,GoldenEye (1995),Action|Adventure|Thriller,0.553065,0.629522,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.545730,0.0,0.0,1995


In [97]:
movies = pd.get_dummies(movies, columns=['year'])

In [103]:
to_train = movies.columns

In [107]:
to_train = [s for i, s in enumerate(movies.columns) if i > 1]

In [109]:
to_train_df = movies[to_train]

In [110]:
to_train_df

,g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,...,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,0.000000,0.416846,0.516225,0.504845,0.267586,0.000000,0.0,0.000000,0.482990,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.000000,0.512361,0.000000,0.620525,0.000000,0.000000,0.0,0.000000,0.593662,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.000000,0.000000,0.000000,0.000000,0.570915,0.000000,0.0,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0.000000,0.000000,0.000000,0.000000,0.505015,0.000000,0.0,0.466405,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
5,0.549328,0.000000,0.000000,0.000000,0.000000,0.635947,0.0,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
6,0.000000,0.000000,0.000000,0.000000,0.570915,0.000000,0.0,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
7,0.000000,0.636699,0.000000,0.771112,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
8,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
9,0.553065,0.629522,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
